## 4단계: 수치 미분

### 4.1 미분이란

미분은 '변화율'이다.

미분을 러프하게 수식으로 표현하면 $f(x)$에 대해 다음과 같이 정의된다.

$$f'(x) = \lim_{h\rightarrow 0}\frac{f(x+h)-f(x)}{h}$$

이 때, $f'(x)$를 $f(x)$의 도함수라고 한다.

<img src="images/그림 4-1.png" width=500/>

### 4.2 수치 미분 구현

위 식을 참고하여 미분을 계산하는 코드를 구현해보자.

실제 극한을 취급할 수는 없으니 $h$를 극한과 비슷한 값으로 대체하자. ($h = 0.0001$)

이렇게 미세한 값으로 함수의 변화량을 구하는 방법을 수치 미분(numerical differentiation)이라고 한다.

하지만 수치 미분은 작은 $h$를 이용하여 '진정한 미분'을 근사하기 때문에 필연적으로 오차가 포함된다.

이를 완화하기 위해 '중앙차분(centered differnece)'를 이용할 수 있다.

중앙차분은 $f(x+h)$와 $f(x-h)$의 차이를 이용한다.

식은 아래와 같다. (분모가 $2h$인 점 주의)

$$f'(x)=\lim_{h\rightarrow 0}\frac{f(x+h)-f(x-h)}{2h}$$
<br>
<img src="images/그림 4-2.png" width=500/>

In [1]:
import numpy as np

class Variable:
    def __init__(self, data):
        self.data = data

class Function:
    def __call__(self, input: Variable):
        x = input.data
        y = self.forward(x) # 구체적인 계산은 forward 메서드에서 한다.
        output = Variable(y)
        return output

    def forward(self, x):
        raise NotImplementedError

class Square(Function):
    def forward(self, x):
        return x ** 2

class Exp(Function):
    def forward(self, x):
        return np.exp(x)

In [2]:
# 수치 미분의 구현
def numerical_diff(f, x, eps=1e-4):
    # eps: epsilon으로, 매우 작은 h를 나타냄
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    # 중앙차분을 이용한 수치 미분
    return (y1.data - y0.data) / (2 * eps)

f = Square()
x = Variable(np.array(2.0))
dy = numerical_diff(f, x)
print(dy)

4.000000000004


미분은 해석적으로 계산할 수도 있는데, 해석적 계산은 우리에게 익숙한 '수식 변형을 통한 미분'을 가리킨다.

위의 예시에서는 $y=x^2$일 때 $\frac{dy}{dx}=2x$가 되고,

$x=2.0$에서 미분값은 $4.0$이 된다.

앞의 수치 미분 값과 비교하면 수치 미분의 오차가 매우 작음을 알 수 있다.

### 4.3 합성 함수의 미분

$y=(e^{x^2})^2$를 미분해보자.

In [3]:
def f(x):
    A = Square()
    B = Exp()
    C = Square()
    return C(B(A(x)))

x = Variable(np.array(0.5))
dy = numerical_diff(f, x)
print(dy)

3.2974426293330694


이를 해석적으로 계산한 결과와 비교해보면 다음과 같다. (부동소수점 오차 감안 필요)

In [4]:
# (f(g(h))))' = f'(g(h))*g(h) * g'(h)*h * h'
analytic_dy = (2 * np.exp(x.data**2) * np.exp(x.data**2) * 2*x.data).data
print(f"오차: {analytic_dy - dy:.12f}")

오차: -0.000000087933


### 4.4 수치 미분의 문제점

수치 미분의 결과값에는 오차가 존재하고, 어떤 계산이냐에 따라 오차가 커질 수 있다.

또한, 변수가 여러 개인 계산을 미분하는 경우 각각을 수치 미분하면 계산량이 많아진다.

특히 수백만에서 수백억 개의 매개변수를 이용하는 신경망에서는 수치 미분은 현실적이지 않다.

이를 다음 단계에서 역전파로 해결해보자.

덧붙여서, 수치 미분은 구현이 쉽고 거의 정확한 값을 얻을 수 있기 때문에 역전파 알고리즘의 테스트에 사용되기도 한다.

이를 기울기 확인(gradient checking)이라고 하는데, 단순히 수치 미분 결과와 역전파 결과를 비교하는 것이다.

이는 10단계 "테스트"에서 구현한다.